In [ ]:
from b3d.chisight.dynamic_object_model.dynamic_object_model import dynamic_object_generative_model, viz_trace, info_from_trace
from b3d.chisight.dynamic_object_model.dynamic_object_inference import inference_step
b3d.reload(b3d.chisight.dynamic_object_model.dynamic_object_model)
b3d.reload(b3d.chisight.dynamic_object_model.dynamic_object_inference)

In [ ]:
### Set up model ###

T = 0
b3d.rr_set_time(T)
OBJECT_INDEX = 5

template_pose = all_data[T]["camera_pose"].inv() @ all_data[T]["object_poses"][OBJECT_INDEX]
rendered_rgbd = renderer.render_rgbd_from_mesh(meshes[OBJECT_INDEX].transform(template_pose))
xyz_rendered = b3d.xyz_from_depth(rendered_rgbd[..., 3], fx, fy, cx, cy)

fx, fy, cx, cy = all_data[T]["camera_intrinsics"]
xyz_observed = b3d.xyz_from_depth(all_data[T]["rgbd"][..., 3], fx, fy, cx, cy)
mask = all_data[T]["masks"][OBJECT_INDEX] * (xyz_observed[..., 2] > 0) * (jnp.linalg.norm(xyz_rendered - xyz_observed, axis=-1) < 0.01)
model_vertices = template_pose.inv().apply(xyz_rendered[mask])
model_colors = vertex_attributes=all_data[T]["rgbd"][..., :3][mask]

hyperparams = {
    "vertices": model_vertices,
    "max_color_shift": Pytree.const(0.05),
    "max_pose_position_shift": Pytree.const(0.1),
    "fx": fx,
    "fy": fy,
    "cx": cx,
    "cy": cy,
    "image_height": Pytree.const(image_height),
    "image_width": Pytree.const(image_width)
}

previous_state = {
    "pose": template_pose,
    "colors": model_colors,
}

choicemap = genjax.ChoiceMap.d(
    {
        "pose": template_pose,
        "colors": model_colors,

        "rgbd": all_data[T]["rgbd"],

        "color_variance": 0.05,
        "depth_variance": 0.005,
        "color_outlier_probability": jnp.ones(len(model_vertices)) * 0.01,
        "depth_outlier_probability": jnp.ones(len(model_vertices)) * 0.01,
    }
)
key = jax.random.PRNGKey(0)
trace, _ = dynamic_object_generative_model.importance(key, choicemap, (hyperparams, previous_state))
viz_trace(trace, 0)
results = {}
info = info_from_trace(trace)
trace.get_score()

In [ ]:
### Run inference ###
for T in tqdm(range(len(all_data))):
    trace = inference_step(trace, key, all_data[T]["rgbd"])
    viz_trace(trace, T)
    results[T] = trace

viz_trace(trace, T, ground_truth_vertices=meshes[OBJECT_INDEX].vertices, ground_truth_pose=all_data[T]["camera_pose"].inv() @ all_data[T]["object_poses"][OBJECT_INDEX])

In [ ]:
info_from_trace(trace)["scores"] < 0.0

In [304]:
info_from_trace(trace)["scores"] < 0.0

Array([ 1.8246567 ,  0.98210424, -0.11121082, ..., -8.72367   ,
        2.7278347 ,  2.7573478 ], dtype=float32)